# Federated PyTorch MNIST Tutorial

In [1]:
#Install dependencies if not already installed
!pip install torch torchvision

In [2]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
import torchvision.transforms as transforms
import openfl.native as fx
from openfl.federated import FederatedModel,FederatedDataSet


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


After importing the required packages, the next step is setting up our openfl workspace. To do this, simply run the `fx.init()` command as follows:

In [3]:
#Setup default workspace, logging, etc.
fx.init('torch_cnn_mnist', log_level='METRIC', log_file='./spam_metric.log')

Creating Workspace Directories
Creating Workspace Templates



New workspace directory structure:
workspace
├── data
│   └── MNIST
│       └── raw
├── src
│   ├── mnist_utils.py
│   ├── __pycache__
│   │   ├── mnist_utils.cpython-310.pyc
│   │   └── __init__.cpython-310.pyc
│   ├── ptmnist_inmemory.py
│   ├── pt_cnn.py
│   └── __init__.py
├── cert
├── save
│   ├── torch_cnn_mnist_last.pbuf
│   ├── torch_cnn_mnist_best.pbuf
│   └── torch_cnn_mnist_init.pbuf
├── final_pytorch_model
├── agg_to_col_two_signed_cert.zip
├── spam_metric.log
├── .workspace
├── plan
│   ├── defaults
│   ├── cols.yaml
│   ├── plan.yaml
│   ├── plan_evaluation.yaml
│   └── data.yaml
├── agg_to_col_one_signed_cert.zip
├── requirements.txt
└── logs
    └── cnn_mnist
        ├── events.out.tfevents.1682679019.241369ca6574.25569.1
        └── events.out.tfevents.1682679019.241369ca6574.25569.0

10 directories, 22 files
Setting Up Certificate Authority...

1.  Create Root CA
1.1 Create Directories
1.2 Create Database
1.3 Create CA Request and Certificate
2.  Create Signing Certi

Now we are ready to define our dataset and model to perform federated learning on. The dataset should be composed of a numpy arrayWe start with a simple fully connected model that is trained on the MNIST dataset. 

In [4]:
def one_hot(labels, classes):
    return np.eye(classes)[labels]

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)

train_images,train_labels = trainset.train_data, np.array(trainset.train_labels)
train_images = torch.from_numpy(np.expand_dims(train_images, axis=1)).float()

validset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)

valid_images,valid_labels = validset.test_data, np.array(validset.test_labels)
valid_images = torch.from_numpy(np.expand_dims(valid_images, axis=1)).float()
valid_labels = one_hot(valid_labels,10)

/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:65: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/usr/local/lib/python3.10/dist-packages/torchvision/datasets/mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


In [5]:
feature_shape = train_images.shape[1]
classes       = 10

fl_data = FederatedDataSet(train_images,train_labels,valid_images,valid_labels,batch_size=32,num_classes=classes)

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(16, 32, 3)
        self.fc1 = nn.Linear(32 * 5 * 5, 32)
        self.fc2 = nn.Linear(32, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return F.log_softmax(x, dim=1)
    
optimizer = lambda x: optim.Adam(x, lr=1e-4)

def cross_entropy(output, target):
    """Binary cross-entropy metric
    """
    return F.cross_entropy(input=output,target=target)

Here we can define metric logging function. It should has the following signature described below. You can use it to write metrics to tensorboard or some another specific logging.

In [6]:
from torch.utils.tensorboard import SummaryWriter

writer = SummaryWriter('./logs/cnn_mnist', flush_secs=5)


def write_metric(node_name, task_name, metric_name, metric, round_number):
    writer.add_scalar("{}/{}/{}".format(node_name, task_name, metric_name),
                      metric, round_number)

In [7]:

#Create a federated model using the pytorch class, lambda optimizer function, and loss function
fl_model = FederatedModel(build_model=Net,optimizer=optimizer,loss_fn=cross_entropy,data_loader=fl_data)

[10:53:41] WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=665583;file:///usr/local/lib/python3.10/dist-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=126564;file:///usr/local/lib/python3.10/dist-packages/openfl/utilities/utils.py#172\172]8;;\

The `FederatedModel` object is a wrapper around your Keras, Tensorflow or PyTorch model that makes it compatible with openfl. It provides built in federated training and validation functions that we will see used below. Using it's `setup` function, collaborator models and datasets can be automatically defined for the experiment. 

In [8]:
collaborator_models = fl_model.setup(num_collaborators=3)
train_test_collaborators = {'one':collaborator_models[0],'two':collaborator_models[1]}#, 'three':collaborator_models[2]}
validate_collaborators = {'three': collaborator_models[2]}

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=130935;file:///usr/local/lib/python3.10/dist-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=517497;file:///usr/local/lib/python3.10/dist-packages/openfl/utilities/utils.py#172\172]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=148058;file:///usr/local/lib/python3.10/dist-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=787042;file:///usr/local/lib/python3.10/dist-packages/openfl/utilities/utils.py#172\172]8;;\

           WARNING  tried to remove tensor: __opt_state_needed not present in the tensor dict                                                       ]8;id=774914;file:///usr/local/lib/python3.10/dist-packages/openfl/utilities/utils.py\utils.py]8;;\:]8;id=500717;file:///usr/local/lib/python3.10/dist-packages/openfl/utilities/utils.py#172\172]8;;\

In [9]:
#Original MNIST dataset
print(f'Original training data size: {len(train_images)}')
print(f'Original validation data size: {len(valid_images)}\n')

#Collaborator one's data
print(f'Collaborator one\'s training data size: {len(collaborator_models[0].data_loader.X_train)}')
print(f'Collaborator one\'s validation data size: {len(collaborator_models[0].data_loader.X_valid)}\n')

#Collaborator two's data
print(f'Collaborator two\'s training data size: {len(collaborator_models[1].data_loader.X_train)}')
print(f'Collaborator two\'s validation data size: {len(collaborator_models[1].data_loader.X_valid)}\n')

# Collaborator three's data
print(f'Collaborator three\'s training data size: {len(collaborator_models[2].data_loader.X_train)}')
print(f'Collaborator three\'s validation data size: {len(collaborator_models[2].data_loader.X_valid)}')

Original training data size: 60000
Original validation data size: 10000

Collaborator one's training data size: 20000
Collaborator one's validation data size: 3334

Collaborator two's training data size: 20000
Collaborator two's validation data size: 3333

Collaborator three's training data size: 20000
Collaborator three's validation data size: 3333


We can see the current plan values by running the `fx.get_plan()` function

In [10]:
 #Get the current values of the plan. Each of these can be overridden
print(fx.get_plan())

{
    "aggregator.settings.best_state_path": "save/torch_cnn_mnist_best.pbuf",
    "aggregator.settings.db_store_rounds": 2,
    "aggregator.settings.init_state_path": "save/torch_cnn_mnist_init.pbuf",
    "aggregator.settings.last_state_path": "save/torch_cnn_mnist_last.pbuf",
    "aggregator.settings.log_metric_callback.template": "src.mnist_utils.write_metric",
    "aggregator.settings.rounds_to_train": 10,
    "aggregator.settings.write_logs": true,
    "aggregator.template": "openfl.component.Aggregator",
    "assigner.settings.task_groups.0.name": "train_and_validate",
    "assigner.settings.task_groups.0.percentage": 1.0,
    "assigner.settings.task_groups.0.tasks.0": "aggregated_model_validation",
    "assigner.settings.task_groups.0.tasks.1": "train",
    "assigner.settings.task_groups.0.tasks.2": "locally_tuned_model_validation",
    "assigner.template": "openfl.component.RandomGroupedAssigner",
    "collaborator.settings.db_store_rounds": 1,
    "collaborator.settings.delta_

Now we are ready to run our experiment. If we want to pass in custom plan settings, we can easily do that with the `override_config` parameter

In [11]:
# Run experiment, return trained FederatedModel

final_fl_model = fx.run_experiment(train_test_collaborators, override_config={
    'aggregator.settings.rounds_to_train': 2,
    'aggregator.settings.log_metric_callback': write_metric,
})

/usr/local/lib/python3.10/dist-packages/openfl/federated/task/runner_pt.py:284: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at ../torch/csrc/utils/tensor_numpy.cpp:199.)
  new_state[k] = pt.from_numpy(tensor_dict.pop(k)).to(device)
/usr/local/lib/python3.10/dist-packages/openfl/federated/task/runner_pt.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


[10:53:42] METRIC   Round 0, collaborator one is sending metric for task aggregated_model_validation: acc   0.118476                         ]8;id=689693;file:///usr/local/lib/python3.10/dist-packages/openfl/component/collaborator/collaborator.py\collaborator.py]8;;\:]8;id=76793;file:///usr/local/lib/python3.10/dist-packages/openfl/component/collaborator/collaborator.py#415\415]8;;\

           METRIC   Round 0, collaborator validate_agg aggregated_model_validation result acc:      0.118476                                   ]8;id=572022;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=54946;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py#559\559]8;;\

/usr/local/lib/python3.10/dist-packages/openfl/federated/task/runner_pt.py:458: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


[10:53:43] METRIC   Round 0, collaborator one is sending metric for task train: cross_entropy       0.634349                                 ]8;id=851752;file:///usr/local/lib/python3.10/dist-packages/openfl/component/collaborator/collaborator.py\collaborator.py]8;;\:]8;id=415463;file:///usr/local/lib/python3.10/dist-packages/openfl/component/collaborator/collaborator.py#415\415]8;;\

...

           METRIC   Round 1, collaborator validate_local locally_tuned_model_validation result acc: 0.952895                                   ]8;id=817447;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=913965;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py#559\559]8;;\

           METRIC   Round 1, aggregator: train <openfl.interface.aggregation_functions.weighted_average.WeightedAverage object at              ]8;id=747054;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=45321;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    0x7f3c337ff760> cross_entropy:    0.178438                                                                                                  

           METRIC   Round 1, aggregator: locally_tuned_model_validation                                                                        ]8;id=445820;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=434712;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    <openfl.interface.aggregation_functions.weighted_average.WeightedAverage object at 0x7f3c337ff760> acc:     0.960552                        

           METRIC   Round 1, aggregator: aggregated_model_validation <openfl.interface.aggregation_functions.weighted_average.WeightedAverage  ]8;id=828316;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=175548;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    object at 0x7f3c337ff760> acc:        0.936103                                                                                              

[10:53:48] METRIC   Round 1: saved the best model with score 0.936103                                                                          ]8;id=946380;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=373163;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py#858\858]8;;\

In [12]:
#Save final model
final_fl_model.save_native('final_pytorch_model')

Now we have finished the training process.

We can validate the model on `validate_collaborators`. It's worth to noting that we can get the tensor dict with `get_tensor_dict()` from `final_fl_model`. 

First, we setup the new aggregator's tensors with the tensors of the trained model(`final_fl_model`). Then, we set the parameter 'rounds_to_train' to 1 to cancel the training process. Finally, we only let collaborators perform "aggregated_model_validation" task to achieve federated evaluation.

In [14]:
tensor_dict = final_fl_model.get_tensor_dict()

fx.run_evaluation(validate_collaborators, update_plan_file='plan/plan_evaluation.yaml', aggregator_tensor_dict=tensor_dict)

/usr/local/lib/python3.10/dist-packages/openfl/federated/task/runner_pt.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data, target = pt.tensor(data).to(self.device), pt.tensor(


[10:55:16] METRIC   Round 0, collaborator three is sending metric for task aggregated_model_validation: acc 0.936994                         ]8;id=206687;file:///usr/local/lib/python3.10/dist-packages/openfl/component/collaborator/collaborator.py\collaborator.py]8;;\:]8;id=890079;file:///usr/local/lib/python3.10/dist-packages/openfl/component/collaborator/collaborator.py#415\415]8;;\

           METRIC   Round 0, collaborator validate_agg aggregated_model_validation result acc:      0.936994                                   ]8;id=941998;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=246216;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py#559\559]8;;\

           METRIC   Round 0, aggregator: aggregated_model_validation <openfl.interface.aggregation_functions.weighted_average.WeightedAverage  ]8;id=145571;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=938784;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py#842\842]8;;\
                    object at 0x7f3c337ff760> acc:        0.936994                                                                                              

           METRIC   Round 0: saved the best model with score 0.936994                                                                          ]8;id=250840;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py\aggregator.py]8;;\:]8;id=603242;file:///usr/local/lib/python3.10/dist-packages/openfl/component/aggregator/aggregator.py#858\858]8;;\